In [1]:
import numpy as np
import random
import math
import pandas as pd
import matplotlib.pyplot as plt

''' Provides an abstraction of a data point as a Euclidean point with (x,y) coordinates. '''
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return '(' + str(self.x) + ',' + str(self.y) + ')'
    
    def __repr__(self):
        return self.__str__()
    
    ''' Returns the point in the given list that is closest (in terms of Euclidean distance) to this point. '''
    def closest_point(self, points):
        shortest_dist = float('inf')
        closest_point = 0
        for point in points:
            dist = math.sqrt((point.x - self.x)**2 + (point.y - self.y)**2)
            if dist < shortest_dist:
                shortest_dist = dist
                closest_point = point
        return closest_point

In [18]:
''' Returns the point that is closest to the average of the given list of Point objects. '''
def average_point(points):
    avg_x = 0.
    avg_y = 0.
    for point in points:
        avg_x += point.x
        avg_y += point.y
    avg_x /= len(points)
    avg_y /= len(points)
    
    avgPt = Point(avg_x, avg_y)
    closestToAvg = avgPt.closest_point(points)
    return closestToAvg
    #return avgPt

In [19]:
''' Creates a scatterplot of the given clusters, color-coding by cluster and highlighting centroids in red. '''
def plot_clusters(points, clusters):
    colors = ['blue', 'green', 'cyan', 'magenta', 'yellow', 'black']
    centroids = clusters.keys()
    
    for point in points:
        if point in centroids:
            plt.scatter(point.x, point.y, color='red')
        else:
            for centroid in clusters:
                if point in clusters[centroid]:
                    colorInd = centroids.index(centroid) % len(colors)
            plt.scatter(point.x, point.y, color=colors[colorInd])
    plt.show()

In [20]:
''' K medioids centroid initialization. '''
def init_k_medioids(points, k):
    return random.sample(points, k)

In [21]:
''' Kmeans++ centroid initialization. '''
def init_kpp(points, k):
    # Choose first centroid uniformly at random
    centroids = [random.choice(points)]
    
    # Choose the remaining k-1 centroids
    for i in range(1, k):
        # For each point, calculate the squared distance between it and the closest already-chosen centroid
        distances = [None for j in range(len(points))]
        for j in range(len(points)):
            closest_cent = points[j].closest_point(centroids)
            distances[j] = (closest_cent.x - points[j].x)**2 + (closest_cent.y - points[j].y)**2

        # Generate probability weights proportional to the previously calculated squared distances
        weights = [distances[j] / sum(distances) for j in range(len(distances))]

        # Choose a new centroid randomly from the remaining points 
        centroids.append(np.random.choice(points, p=weights))

    return centroids

In [22]:
''' Performs the k-means clustering algorithm using the given centroid initialization method. '''
def k_means(centroid_init_func, points, k):
    # Initialize the centroids by randomly choosing k points from the dataset
    centroids = centroid_init_func(points, k)
    previous_centroids = []
    
    # Assign clusters and reassign centroids until they stop changing
    while centroids != previous_centroids:
        previous_centroids = [cent for cent in centroids]
        clusters = {cent: [cent] for cent in centroids}
        
        # Assign each point to the closest centroid
        for point in points:
            if point not in centroids:
                # Find the centroid closest to the point and add it to its cluster
                closest_cent = point.closest_point(centroids)
                clusters[closest_cent].append(point)
                
        # Plot the current clusters
        #plot_clusters(points, clusters)
                
        # Keep track of the previous set of centroids
        last_centroids = centroids

        # Reassign centroids 
        for centroid in clusters:
            centroids.append(average_point(clusters[centroid]))    # Assign the new centroid
            centroids.remove(centroid)    # Delete the old centroid
    
    # Plot the final clusters
    plot_clusters(points, clusters)

In [23]:
''' Plots sepal width vs. sepal length from Iris Data. '''
def plot_iris_data():
    data = pd.read_csv('/Users/anthonydepalatis/Documents/School/EE379K/Lab6/data/iris_data.csv')
    plt.scatter(data['Sepal.Length'], data['Sepal.Width'])
    plt.title('Sepal Width vs. Sepal Length')
    plt.xlabel('Sepal Length')
    plt.ylabel('Sepal Width')
    plt.show()

In [ ]:
mList = [Point(round(np.random.uniform(0,5), 2), round(np.random.uniform(0,5), 2)) for i in range(100)]
k_means(init_k_medioids, mList, 6)
plot_iris_data()

data = pd.read_csv('/Users/anthonydepalatis/Documents/School/EE379K/Lab6/data/iris_data.csv')
length = data['Sepal.Length']
width = data['Sepal.Width']

points = []
for x in range(len(length)):
    points.append(Point(length[x], width[x]))

#for k in range(2, 6):
#    k_means(init_k_medioids, points, k)